# Optimizing DataFrames and Processing in Chunks

## Introduction

In this project, we will focus on working with chunked dataframes in Pandas while also optimising the dataframe's memory usage.

The dataset we'll be working with contains financial lending data from [Lending Club](https://www.lendingclub.com/), a marketplace for personal loans that matches borrowers with investors.
The Lending Club's website lists aprroved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other application details. Once the lender is ready to back a loan, it selects the amount off money they want to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the [origination fee](https://help.lendingclub.com/hc/en-us/articles/214463677) that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011. In total, the entire dataset consumes about 67 megabytes of memory. For the purposes of our project, we will imagine that we only have 10 megabytes of memory available on our machine.

Let's explore the dataset and look for any data quality issues.


In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 99

first_five_rows = pd.read_csv('loans_2007.csv', nrows=5)
print(first_five_rows)

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade                 emp_title emp_length  \
0   10.65%       162.87     B        B2                       NaN  10+ years   
1   15.27%        59.83     C        C4                     Ryder   < 1 year   
2   15.96%        84.33     C        C5                       NaN  10+ years   
3   13.49%       339.31     C        C1       AIR RESOURCES BOARD  10+ years   
4   12.69%        67.79     B        B5  University Medical Group     1 year   

  home_ownership  annual_inc verification_status  

There are no glaring data quality issues. However, there are a few opportunities to optimize the data.

Now let's decide on the optimal chunk size to use when working with our dataset. Ideally, we will aim to adjust the number of rows so that the memory usage is just under 5 megabytes. This is to allow us an overhead in order to perform calculations using the dataframes.

In [2]:
first_1000 = pd.read_csv('loans_2007.csv', nrows=1000)
print(first_1000.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          1000 non-null   int64  
 1   member_id                   1000 non-null   float64
 2   loan_amnt                   1000 non-null   float64
 3   funded_amnt                 1000 non-null   float64
 4   funded_amnt_inv             1000 non-null   float64
 5   term                        1000 non-null   object 
 6   int_rate                    1000 non-null   object 
 7   installment                 1000 non-null   float64
 8   grade                       1000 non-null   object 
 9   sub_grade                   1000 non-null   object 
 10  emp_title                   949 non-null    object 
 11  emp_length                  983 non-null    object 
 12  home_ownership              1000 non-null   object 
 13  annual_inc                  1000 n

As we can see, 1000 rows consume approx. 1.5 MB. Reading in the file is more efficient if we utilize as much memory as we can. Therefore, we will use chunksizes of 3000 rows, which will use approx. 4.5 MB.
Let's verify this:

In [3]:
loan_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in loan_chunks:
    print(chunk.memory_usage(deep=True).sum() / 2**20)

4.580394744873047
4.576141357421875
4.577898979187012
4.579251289367676
4.575444221496582
4.577326774597168
4.575918197631836
4.578287124633789
4.576413154602051
4.57646369934082
4.589176177978516
4.588043212890625
4.594850540161133
4.828314781188965
0.868586540222168


## Exploring the Data in Chunks

Let's familiarise ourselves with the columns to see which ones we can optimise, while working with the data using chunks.

In [4]:
# How many columns have a numeric type? How many have a string type?
loan_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)

num_numeric = []
num_string = []
for chunk in loan_chunks:
    total_numeric_cols = chunk.select_dtypes(include=[np.number]).shape[1]
    num_numeric.append(total_numeric_cols)
    total_string_cols = chunk.select_dtypes(include=['object']).shape[1]
    num_string.append(total_string_cols)

print(num_numeric)
print(num_string)
    

    

[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


In [5]:
# How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
loan_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)

unique_values = {}
for chunk in loan_chunks:
    string_cols = chunk.select_dtypes(include='object')
    for col in string_cols.columns:
        col_unique_values = string_cols[col].value_counts()
        if col in unique_values:
            unique_values[col].append(col_unique_values)
        else:
            unique_values[col] = [col_unique_values]

combined_unique_values = {}

for col in unique_values:
    u_concat = pd.concat(unique_values[col])
    u_group = u_concat.groupby(u_concat.index).sum()
    col_unique_count = len(u_group)
    col_total_count = sum(u_group)
    unique_percentage = (col_unique_count / col_total_count) * 100
    combined_unique_values[col] = {'column': col, 'unique values': col_unique_count, 'percentage unique': unique_percentage, 'convert to category': unique_percentage < 50}
    
    print(combined_unique_values[col])
    print('\n')

 


{'column': 'term', 'unique values': 2, 'percentage unique': 0.004702010109321735, 'convert to category': True}


{'column': 'int_rate', 'unique values': 394, 'percentage unique': 0.9262959915363819, 'convert to category': True}


{'column': 'grade', 'unique values': 7, 'percentage unique': 0.01645703538262607, 'convert to category': True}


{'column': 'sub_grade', 'unique values': 35, 'percentage unique': 0.08228517691313036, 'convert to category': True}


{'column': 'emp_title', 'unique values': 30658, 'percentage unique': 76.81976496529605, 'convert to category': False}


{'column': 'emp_length', 'unique values': 11, 'percentage unique': 0.026555295367308017, 'convert to category': True}


{'column': 'home_ownership', 'unique values': 5, 'percentage unique': 0.011755025273304338, 'convert to category': True}


{'column': 'verification_status', 'unique values': 3, 'percentage unique': 0.007053015163982603, 'convert to category': True}


{'column': 'issue_d', 'unique values': 55, 'perc

The `emp_title`, `title` and `id` columns each have more than 50% of values that are unique. This means that they are not suitable to be converted into the category datatype.
However, coverting the remaining columns into category datatype may help us save on memory space and improve speed.

In [6]:
# Which float columns have no missing values and could be candidates for conversion into the integer type?

loan_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)


missing = []
for chunk in loan_chunks:
    floats = chunk.select_dtypes(include='float')
    missing.append(floats.apply(pd.isnull).sum())
    

combined_missing = pd.concat(missing)
grouped_missing = combined_missing.groupby(combined_missing.index).sum().sort_values()

    
print(grouped_missing)

            
        
        

member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
total_pymnt                      3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
loan_amnt                        3
last_pymnt_amnt                  3
total_rec_prncp                  3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
collection_recovery_fee          3
installment                      3
annual_inc                       7
inq_last_6mths                  32
total_acc                       32
delinq_2yrs                     32
pub_rec                         32
delinq_amnt                     32
open_acc                        32
acc_now_delinq                  32
tax_liens                      108
collections_12_mths_ex_med     148
chargeoff_within_12_

We can see that all of the float columns have missing values. This means that we cannot convert them to integer types, as they do not allow missing values.

In [7]:
# Calculate the total memory usage across all chunks

loan_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)

total_memory = 0

for chunk in loan_chunks:
    memory = chunk.memory_usage(deep=True).sum()
    total_memory += memory

total_memory = total_memory / 2**20 # Convert to megabytes

print(total_memory)
    


65.24251079559326



## Optimising String Columns

The best way to achieve substantial memory improvements in our dataframe is to convert the string columns into numeric types wherever possible. We can also optimise the string columns by converting all of the ones we identified as having unique values less than 50% to the category type.

If we look back up to where we printed the first 5 rows of the dataset, we can see that the columns `int_rate` and `revol_util` could be converted into numerical types if they are cleaned to remove the % signs. Also, we have seen that the `term` column only has two unique values: `36 months` and `60 months`. This means we could convert the column into a numerical type as long as we make it clear that the units are months.

Finally, we can convert any columns that contain information relating to dates into the datetime type. This includes the `issue_d`, `earliest_cr_line`, `last_pymnt_d` and `last_credit_pull_d` columns.

In order to keep track of the changes we will build upon the code incrementally. This will also ensure the changes are maintained as we go along.

In [8]:
# Convert date columns to datetime type
chunk_itr = pd.read_csv('loans_2007.csv', chunksize=3000, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

chunk.dtypes

id                             object
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc    

In [9]:
chunk_itr = pd.read_csv('loans_2007.csv', chunksize=3000, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

# Remove unwanted characters and convert to numeric types
for chunk in chunk_itr:
    int_cleaned = chunk['int_rate'].str.rstrip('%')
    revol_cleaned = chunk['revol_util'].str.rstrip('%')
    term_cleaned = chunk['term'].str.rstrip('months')
    
    chunk['int_rate'] = pd.to_numeric(int_cleaned, downcast='float')
    chunk['revol_util'] = pd.to_numeric(revol_cleaned, downcast='float')
    chunk['term'] = pd.to_numeric(term_cleaned, downcast='float')

chunk.dtypes

id                                    object
member_id                            float64
loan_amnt                            float64
funded_amnt                          float64
funded_amnt_inv                      float64
term                                 float32
int_rate                             float32
installment                          float64
grade                                 object
sub_grade                             object
emp_title                             object
emp_length                            object
home_ownership                        object
annual_inc                           float64
verification_status                   object
issue_d                       datetime64[ns]
loan_status                           object
pymnt_plan                            object
purpose                               object
title                                 object
zip_code                              object
addr_state                            object
dti       

In [10]:
chunk_itr = pd.read_csv('loans_2007.csv', chunksize=3000, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

convert_col_dtypes = {}

for chunk in chunk_itr:
    int_cleaned = chunk['int_rate'].str.rstrip('%')
    revol_cleaned = chunk['revol_util'].str.rstrip('%')
    term_cleaned = chunk['term'].str.rstrip('months')
    
    chunk['int_rate'] = pd.to_numeric(int_cleaned, downcast='float')
    chunk['revol_util'] = pd.to_numeric(revol_cleaned, downcast='float')
    chunk['term'] = pd.to_numeric(term_cleaned, downcast='float')
    
# Assess remaining string types to decide whether they should be converted to category types. Compile a dictionary of columns to convert as we read in the csv in the next iteration.
    
    strings = chunk.select_dtypes(include='object')
    for col in strings.columns:
        if combined_unique_values[col]['convert to category'] and col not in convert_col_dtypes: # lookup column in analysis carried out above to assess whether it can be converted to category
            convert_col_dtypes[col] = 'category'

print(convert_col_dtypes)

{'grade': 'category', 'sub_grade': 'category', 'emp_length': 'category', 'home_ownership': 'category', 'verification_status': 'category', 'loan_status': 'category', 'pymnt_plan': 'category', 'purpose': 'category', 'zip_code': 'category', 'addr_state': 'category', 'initial_list_status': 'category', 'application_type': 'category'}


In [11]:
chunk_itr = pd.read_csv('loans_2007.csv', chunksize=3000, dtype=convert_col_dtypes, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

total_memory = 0

for chunk in chunk_itr:
    int_cleaned = chunk['int_rate'].str.rstrip('%')
    revol_cleaned = chunk['revol_util'].str.rstrip('%')
    term_cleaned = chunk['term'].str.rstrip('months')
    
    chunk['int_rate'] = pd.to_numeric(int_cleaned, downcast='float')
    chunk['revol_util'] = pd.to_numeric(revol_cleaned, downcast='float')
    chunk['term'] = pd.to_numeric(term_cleaned, downcast='float')
    
# Calculate total memory footprint now that the string columns have been converted
    memory = chunk.memory_usage(deep=True).sum()
    total_memory += memory

total_memory = total_memory / 2**20
print(total_memory)


19.393789291381836


We can see that by converting the string columns into more appropriate datatypes, we have reduced the memory footprint of the dataset from 65 MB to 19 MB. That's less than a third!

## Optimising Numeric Columns

Now let's take a look at the numeric columns and see if we can convert any to more appropriate datatypes that will further reduce our memory footprint and runtime. 
We can convert the float columns that contain missing values into more space efficient subtypes. 

We could also convert any float columns that don't contain missing values and also represent whole numbers into the integer type. **However**, in our previous analysis above, we found that every float column had missing values.

In [17]:
chunk_itr = pd.read_csv('loans_2007.csv', chunksize=3000, dtype=convert_col_dtypes, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

total_memory = 0

for chunk in chunk_itr:
    int_cleaned = chunk['int_rate'].str.rstrip('%')
    revol_cleaned = chunk['revol_util'].str.rstrip('%')
    term_cleaned = chunk['term'].str.rstrip('months')
    
    chunk['int_rate'] = pd.to_numeric(int_cleaned, downcast='float')
    chunk['revol_util'] = pd.to_numeric(revol_cleaned, downcast='float')
    chunk['term'] = pd.to_numeric(term_cleaned, downcast='float')
    
    # Convert all float columns into the most efficient subtype
    
    float_columns = chunk.select_dtypes(include='float')
    for col in float_columns.columns:
        chunk[col] = pd.to_numeric(float_columns[col], downcast='float')
    
    # Calculate total memory now numerical types have been optimised
    memory = chunk.memory_usage(deep=True).sum()
    total_memory += memory

total_memory = total_memory / 2**20
print(total_memory)


chunk.dtypes


14.525701522827148


id                                    object
member_id                            float32
loan_amnt                            float32
funded_amnt                          float32
funded_amnt_inv                      float32
term                                 float32
int_rate                             float32
installment                          float32
grade                               category
sub_grade                           category
emp_title                             object
emp_length                          category
home_ownership                      category
annual_inc                           float32
verification_status                 category
issue_d                       datetime64[ns]
loan_status                         category
pymnt_plan                          category
purpose                             category
title                                 object
zip_code                            category
addr_state                          category
dti       

We can see that all of the float columns have been converted into the float32 type. As this datatype uses half as many bits, the memory footprint is reduced substantially.

Converting the float types has further reduced the mb used by the dataframe to 15 MB.